# 初始化环境

In [ ]:
import aie

aie.Authenticate()
aie.Initialize()

# 筛选需要计算处理的矢量区域

In [ ]:
shp = aie.FeatureCollection('China_City') \
         .filter(aie.Filter.eq('city', '杭州市'))
print(shp.getInfo())

In [ ]:
region = shp.geometry()
print(region.getInfo())

# 筛选需要计算处理的影像集合

In [ ]:
l8 = aie.ImageCollection('LANDSAT_LC08_C02_T1_L2') \
        .filterBounds(region) \
        .filterDate('2017-01-01', '2017-12-31') \
        .filter('eo:cloud_cover<20') \
        .limit(5) \
        .select(['SR_B5', 'SR_B4'])

# 计算NDVI指数并进行区域统计

In [ ]:
def ndvi_reduceregion(image):
    dicts = image.normalizedDifference(['SR_B5', 'SR_B4']) \
                 .reduceRegion(aie.Reducer.mean())
    result = aie.Feature(None, dicts)
    result = result.set({'startTime':image.get('meta:startTime')})
    return result

ndvis = l8.map(ndvi_reduceregion)
ndvi_mean = ndvis.getInfo()
print(ndvi_mean)

# 将统计结果按时序绘图

In [ ]:
x_list = []
y_list = []
for feature in ndvi_mean['features']:
    x_list.append(feature['properties']['startTime'])
    y_list.append(feature['properties']['nd_mean'])
    
print(x_list)
print(y_list)

In [ ]:
from bqplot import pyplot as plt
plt.figure(1, title='NDVI Chart')
plt.plot(x_list, y_list)
plt.show()